### Preparação dos Dados

Otimizando as Colunas categóricas

[ FEITO ] Ajustar tipos (ex: definir ‘category’, ‘object’ ou ‘int’ onde precisa)

[ FEITO ] Tratar valores faltantes

[ FEITO ] Mapear variáveis binárias simples (ex: Street) para 0/1

COLUNAS CATEGORICAS

- 'MSZoning' - FEITO
- 'Street' - FEITO
- 'Alley' - FEITO
- 'LotShape' - FEITO
- 'LandContour' - FEITO
- 'Utilities' - N/A
- 'LotConfig' - FEITO
- 'LandSlope' - FEITO - VALIDAR PORQUE ESTA VAZIO 
- 'Neighborhood' - FEITO - OneHot
- 'Condition1' - FEITO Nova coluna - Concat
- 'Condition2' - FEITO Nova coluna - Concat
- 'BldgType' - FEITO - Binário 0-4
- 'HouseStyle' - FEITO - OneHot
- 'RoofStyle'' - FEITO - OneHot
- 'RoofMatl' - FEITO - OneHot
- 'Exterior1st'  - FEITO - OneHot Concat
- 'Exterior2nd'  - FEITO - OneHot Concat
- 'MasVnrType' - FEITO
- 'ExterQual' - FEITO
- 'ExterCond' - FEITO
- 'Foundation' - FEITO - OneHot
- 'BsmtQual' - FEITO - OneHot
- 'BsmtCond'  - FEITO - OneHot
- 'BsmtExposure'  - FEITO - OneHot
- 'BsmtFinType1' - FEITO Nova coluna - Concat
- 'BsmtFinType2' - FEITO Nova coluna - Concat
- 'Heating' - FEITO - OneHot
- 'HeatingQC' - FEITO
- 'CentralAir' - FEITO
- 'Electrical' - FEITO - OneHot
- 'KitchenQual' - FEITO
- 'Functional' - FEITO Nominal
- 'FireplaceQu' - FEITO - Ordinal
- 'GarageType' - FEITO - OneHot
- 'GarageFinish' - FEITO
- 'GarageQual' - FEITO
- 'GarageCond' - FEITO
- 'PavedDrive' - FEITO
- 'PoolQC' - FEITO
- 'Fence' - FEITO
- 'MiscFeature' - FEITO - OneHot
- 'SaleType' - FEITO - OneHot
- 'SaleCondition' - FEITO - OneHot

👇🏻 Terminar
- Utilities | Totalmente desbalanceado | 
- MiscFeature | Categórica nominal, maioria “None”. Use one-hot encoding ou crie variável binária “tem alguma feature extra” (1 = sim, 0 = não), dependendo da relevância. |
- MiscVal | Mesmo caso dade cima |

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
df_train = pd.read_csv('../data/train.csv')

In [7]:
# Criação de um novo df
df_clear = pd.DataFrame({'Id': df_train['Id']})

---

##### Colunas com 2 Categorias - Transformadas em uma coluna binária

In [8]:
# Street
df_clear['Street_encoded'] = df_train['Street'].map({'Pave': 1, 'Grvl':0})

In [9]:
# Alley
df_clear['HasAlley'] = df_train['Alley'].map({'Pave': 1, 'Grvl': 1})
df_clear['HasAlley'] = df_clear['HasAlley'].fillna(0)

In [10]:
# CentralAir
df_clear['CentralAir_encoded'] = df_train['CentralAir'].map({'Y': 1, 'N': 0})

#### Colunas com 3 ou + - Transformadas em uma Matriz binária sem relação ordenada

In [11]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

def add_onehot_column(df_train, df_clear, col_name, drop=None):
    """
    Aplica OneHotEncoder do scikit-learn a uma coluna categórica df_train[col_name]
    e concatena o resultado no df_clear.

    Parâmetros:
    - df_train: DataFrame original com dados
    - df_clear: DataFrame destino onde as novas colunas serão adicionadas
    - col_name: nome da coluna categórica a ser codificada
    - drop: parâmetro do OneHotEncoder para evitar multicolinearidade (ex: 'first', 'if_binary', None)

    Retorna:
    - df_clear com as colunas one-hot adicionadas
    """

    # Criar o encoder
    ohe = OneHotEncoder(sparse_output=False, drop=drop, handle_unknown='ignore')

    # Preparar a coluna para o encoder (reshape 2D) e converter para string para evitar erros com NaNs
    col_array = df_train[col_name].astype(str).values.reshape(-1, 1)

    # Ajustar encoder e transformar
    ohe.fit(col_array)
    col_encoded = ohe.transform(col_array)

    # Criar nomes das colunas no formato "coluna_categoria"
    col_names = [f"{col_name}_{cat}" for cat in ohe.categories_[0]]

    # Criar DataFrame codificado mantendo o índice original para concatenar
    col_df = pd.DataFrame(col_encoded, columns=col_names, index=df_train.index)

    # Concatenar as colunas codificadas no df_clear
    df_clear = pd.concat([df_clear, col_df], axis=1)

    return df_clear

In [12]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

cols = ['BsmtFinType1', 'BsmtFinType2']

# Preencher NaNs por 'Missing'
for col in cols:
    df_train[col] = df_train[col].fillna('Missing')

# OneHotEncoder do scikit-learn
ohe = OneHotEncoder(sparse_output=False, drop=None, handle_unknown='ignore')

# Extrair colunas para codificar e converter para string (prevenção contra erros)
data_to_encode = df_train[cols].astype(str)

# Ajustar e transformar
encoded_array = ohe.fit_transform(data_to_encode)

# Gerar nomes das colunas codificadas no formato "coluna_categoria"
col_names = []
for col, categories in zip(cols, ohe.categories_):
    col_names.extend([f"{col}_{cat}" for cat in categories])

# Criar DataFrame com as novas colunas codificadas
encoded_df = pd.DataFrame(encoded_array, columns=col_names, index=df_train.index)

# Concatenar ao df_clear
df_clear = pd.concat([df_clear, encoded_df], axis=1)

df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Missing,BsmtFinType2_Rec,BsmtFinType2_Unf
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
# MSSubClass 
df_clear = add_onehot_column(df_train, df_clear, 'MSSubClass')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# GarageType
df_train['GarageType'] = df_train['GarageType'].fillna('Missing')
df_clear = add_onehot_column(df_train, df_clear, 'GarageType')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,MSSubClass_80,MSSubClass_85,MSSubClass_90,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_Missing
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# LotShape
df_clear = add_onehot_column(df_train, df_clear, 'LotShape')
df_clear.head()


,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_Missing,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [16]:
# MSZoning
df_clear = add_onehot_column(df_train, df_clear, 'MSZoning')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,GarageType_Missing,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [17]:
# BldgType
df_clear = add_onehot_column(df_train, df_clear, 'BldgType')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [18]:
# LandContour
df_clear = add_onehot_column(df_train, df_clear, 'LandContour')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,MSZoning_RM,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
# LotConfig
df_clear = add_onehot_column(df_train, df_clear, 'LotConfig')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,BldgType_TwnhsE,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [20]:
# LandSlope
df_clear = add_onehot_column(df_train, df_clear, 'LandSlope')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [21]:
# PaveDrive
ohe = OneHotEncoder(sparse_output=False, drop=None, handle_unknown='ignore')
paveddrive_array = df_train['PavedDrive'].astype(str).values.reshape(-1, 1)
ohe.fit(paveddrive_array)
paveddrive_encoded = ohe.transform(paveddrive_array)
col_names = [f"PavedDrive_{cat}" for cat in ohe.categories_[0]]
paveddrive_df = pd.DataFrame(paveddrive_encoded, columns=col_names, index=df_train.index)
df_clear = pd.concat([df_clear, paveddrive_df], axis=1)
df_clear.head()


,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,PavedDrive_N,PavedDrive_P,PavedDrive_Y
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [22]:
# MasVnrType
df_train['MasVnrType'] = df_train['MasVnrType'].fillna('Missing')
df_clear = add_onehot_column(df_train, df_clear, 'MasVnrType')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,PavedDrive_N,PavedDrive_P,PavedDrive_Y,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_Missing,MasVnrType_Stone
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [23]:
# GarageFinish
df_train['GarageFinish'] = df_train['GarageFinish'].fillna('Missing')
df_clear = add_onehot_column(df_train, df_clear, 'GarageFinish')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,PavedDrive_P,PavedDrive_Y,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_Missing,MasVnrType_Stone,GarageFinish_Fin,GarageFinish_Missing,GarageFinish_RFn,GarageFinish_Unf
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [24]:
# Electrical
df_train['Electrical'] = df_train['Electrical'].fillna('Missing')
df_clear = add_onehot_column(df_train, df_clear, 'Electrical')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,GarageFinish_Fin,GarageFinish_Missing,GarageFinish_RFn,GarageFinish_Unf,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Missing,Electrical_Mix,Electrical_SBrkr
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [25]:
# BsmtQual
df_train['BsmtQual'] = df_train['BsmtQual'].fillna('Missing')
df_clear = add_onehot_column(df_train, df_clear, 'BsmtQual')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,Electrical_FuseF,Electrical_FuseP,Electrical_Missing,Electrical_Mix,Electrical_SBrkr,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_Missing,BsmtQual_TA
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [26]:
# BsmtCond
df_train['BsmtCond'] = df_train['BsmtCond'].fillna('Missing')
df_clear = add_onehot_column(df_train, df_clear, 'BsmtCond')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_Missing,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Missing,BsmtCond_Po,BsmtCond_TA
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [27]:
# BsmtCond
df_train['BsmtExposure'] = df_train['BsmtExposure'].fillna('Missing')
df_clear = add_onehot_column(df_train, df_clear, 'BsmtExposure')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Missing,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Missing,BsmtExposure_Mn,BsmtExposure_No
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [28]:
# BsmtCond
df_train['MiscFeature'] = df_train['MiscFeature'].fillna('Missing')
df_clear = add_onehot_column(df_train, df_clear, 'MiscFeature')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Missing,BsmtExposure_Mn,BsmtExposure_No,MiscFeature_Gar2,MiscFeature_Missing,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [29]:
# RoofStyle
df_clear = add_onehot_column(df_train, df_clear, 'RoofStyle')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,MiscFeature_Missing,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [30]:
# Foundation
df_clear = add_onehot_column(df_train, df_clear, 'Foundation')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [31]:
# Functional
df_clear = add_onehot_column(df_train, df_clear, 'Functional')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,Foundation_Slab,Foundation_Stone,Foundation_Wood,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [32]:
# Heating
df_clear = add_onehot_column(df_train, df_clear, 'Heating')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [33]:
# SaleCondition
df_clear = add_onehot_column(df_train, df_clear, 'SaleCondition')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [34]:
# SaleType
df_clear = add_onehot_column(df_train, df_clear, 'SaleType')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,SaleCondition_Partial,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [35]:
# RoofMatl
df_clear = add_onehot_column(df_train, df_clear, 'RoofMatl')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,SaleType_Oth,SaleType_WD,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# HouseStyle
df_clear = add_onehot_column(df_train, df_clear, 'HouseStyle')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,RoofMatl_WdShake,RoofMatl_WdShngl,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [37]:
# Neighborhood
df_clear = add_onehot_column(df_train, df_clear, 'Neighborhood')
df_clear.head()

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Colunas transformadas em variável ordinal

In [38]:
from pandas.api.types import CategoricalDtype

In [39]:
def encode_ordinal(df, col_name):
    """
    Encode coluna categórica ordinal, incluindo Missing para NaNs,
    e retorna a coluna ordinal numérica.
    """
    categories_ordered = ['Po', 'Fa', 'TA', 'Gd', 'Ex']

    cat_type = CategoricalDtype(categories=categories_ordered, ordered=True)
    df[col_name] = df[col_name].astype(cat_type)
    return df[col_name].cat.codes

In [40]:
def encode_ordinal_with_missing(df, col_name, ):
    """
    Encode coluna categórica ordinal, incluindo Missing para NaNs,
    e retorna a coluna ordinal numérica.
    """
    df[col_name] = df[col_name].fillna('Missing').astype(str)

    categories_ordered = ['Missing','Po', 'Fa', 'TA', 'Gd', 'Ex']
    # Adicionar 'Missing' no topo para a ordem ou no fim para evitar interferência
    # Melhor deixar 'Missing' no topo, para indicar ausência = menor qualidade, por exemplo
    if 'Missing' not in categories_ordered:
        categories_ordered = ['Missing'] + categories_ordered

    cat_type = CategoricalDtype(categories=categories_ordered, ordered=True)
    df[col_name] = df[col_name].astype(cat_type)
    return df[col_name].cat.codes

In [41]:
# FireplaceQu
df_clear['FireplaceQu_ordinal'] = encode_ordinal_with_missing(df_train, 'FireplaceQu')
df_clear

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,FireplaceQu_ordinal
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1456,1457,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1457,1458,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
1458,1459,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [42]:
# FireplaceQu
df_clear['GarageQual_ordinal'] = encode_ordinal_with_missing(df_train, 'GarageQual')
df_clear

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,FireplaceQu_ordinal,GarageQual_ordinal
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3,3
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,3
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
1456,1457,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
1457,1458,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,3
1458,1459,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3


In [43]:
# GarageCond
df_clear['GarageCond_ordinal'] = encode_ordinal_with_missing(df_train, 'GarageCond')
df_clear

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,FireplaceQu_ordinal,GarageQual_ordinal,GarageCond_ordinal
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3,3
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3,3,3
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3,3
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,3,3
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3,3
1456,1457,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3,3
1457,1458,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,3,3
1458,1459,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3,3


In [44]:
# ExterQual
df_clear['ExterQual_ordinal'] = encode_ordinal(df_train, 'ExterQual')
df_clear

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,FireplaceQu_ordinal,GarageQual_ordinal,GarageCond_ordinal,ExterQual_ordinal
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,3,3,3
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,3,3,3,2
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,3,3,3
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4,3,3,2
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,3,3,2
1456,1457,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,3,3,2
1457,1458,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4,3,3,4
1458,1459,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,3,3,2


In [45]:
# ExterCond
df_clear['ExterCond_ordinal'] = encode_ordinal(df_train, 'ExterCond')
df_clear

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,FireplaceQu_ordinal,GarageQual_ordinal,GarageCond_ordinal,ExterQual_ordinal,ExterCond_ordinal
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,3,3,3,2
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3,3,3,2,2
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3,3,3,3,2
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4,3,3,2,2
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3,3,3,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3,3,3,2,2
1456,1457,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3,3,3,2,2
1457,1458,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4,3,3,4,3
1458,1459,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,3,3,2,2


In [46]:
# HeatingQC
df_clear['HeatingQC_ordinal'] = encode_ordinal(df_train, 'HeatingQC')
df_clear

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,FireplaceQu_ordinal,GarageQual_ordinal,GarageCond_ordinal,ExterQual_ordinal,ExterCond_ordinal,HeatingQC_ordinal
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,3,3,3,2,4
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,3,3,3,2,2,4
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3,3,3,3,2,4
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4,3,3,2,2,3
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3,3,3,3,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3,3,3,2,2,4
1456,1457,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3,3,3,2,2,2
1457,1458,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4,3,3,4,3,4
1458,1459,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,3,3,2,2,3


In [47]:
# HeatingQC
df_clear['KitchenQual_ordinal'] = encode_ordinal(df_train, 'KitchenQual')
df_clear

,Id,Street_encoded,HasAlley,CentralAir_encoded,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Missing,BsmtFinType1_Rec,...,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,FireplaceQu_ordinal,GarageQual_ordinal,GarageCond_ordinal,ExterQual_ordinal,ExterCond_ordinal,HeatingQC_ordinal,KitchenQual_ordinal
0,1,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,3,3,3,2,4,3
1,2,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,3,3,3,2,2,4,2
2,3,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3,3,3,3,2,4,3
3,4,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4,3,3,2,2,3,3
4,5,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3,3,3,3,2,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3,3,3,2,2,4,2
1456,1457,1,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3,3,3,2,2,2,2
1457,1458,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4,3,3,4,3,4,3
1458,1459,1,0.0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,3,3,2,2,3,3


In [48]:
# LandSlope
land_slope_cat = CategoricalDtype(categories=['Gtl', 'Mod', 'Sev'], ordered=True)
df_clear['LandSlope_ordinal'] = df_train['LandSlope'].astype(land_slope_cat)
df_clear['LandSlope_ordinal'] = df_clear['LandSlope_ordinal'].cat.codes


#### Engenharia de Features

In [49]:
# HasPool
# Criar coluna binária 'HasPool' no df_clear
df_clear['HasPool'] = df_train['PoolQC'].notna().astype(int)

In [50]:
# HasPavement
df_train['HasPavement'] = df_train['PavedDrive'].apply(lambda x: 1 if x in ['Y', 'P'] else 0)


In [51]:
# HasFence
df_train['HasFence'] = df_train['Fence'].notna().astype(int)
df_clear['HasFence'] = df_train['HasFence']

In [52]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

# Criar lista de listas das condições por linha diretamente no df_train
all_conditions = df_train[['Condition1', 'Condition2']].values.tolist()

# Converter para lista de sets para garantir unicidade
all_conditions = [list(set(cond)) for cond in all_conditions]

# Ajustar e transformar
cond_encoded = pd.DataFrame(mlb.fit_transform(all_conditions), columns=mlb.classes_, index=df_train.index)

# Adicionar ao df_clear
df_clear = pd.concat([df_clear, cond_encoded], axis=1)

In [53]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

cols = ['Exterior1st', 'Exterior2nd']

# Inicializa encoder
ohe = OneHotEncoder(sparse_output=False, drop=None, handle_unknown='ignore')

# Extrair e converter para string para garantir segurança
data_to_encode = df_train[cols].astype(str)

# Ajustar e transformar
encoded_array = ohe.fit_transform(data_to_encode)

# Gerar nomes para colunas one-hot
col_names = []
for col, cats in zip(cols, ohe.categories_):
    col_names.extend([f"{col}_{cat}" for cat in cats])

# Criar DataFrame com as colunas codificadas
encoded_df = pd.DataFrame(encoded_array, columns=col_names, index=df_train.index)

# Concatenar ao df_clear
df_clear = pd.concat([df_clear, encoded_df], axis=1)

In [54]:
df_clear.shape

(1460, 234)

#### Validar as colunas

Ver depois

In [55]:
# from sklearn.compose import make_column_transformer
# from sklearn.preprocessing import OneHotEncoder

In [56]:
# Entender esse dado
df_train[df_train['Utilities'] == 'NoSeWa']

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,HasPavement,HasFence
944,945,20,RL,NaN,14375,Pave,NaN,IR1,Lvl,NoSeWa,...,NaN,Missing,0,1,2009,COD,Abnorml,137500,1,0


In [57]:
# Tratar essas duas colunas e verificar o que fazer
mask_na_type = df_train['MasVnrType'].isna()
todos_zero = (df_train.loc[mask_na_type, 'MasVnrArea'] == 0.0).all()
print("Todos os valores ausentes em MasVnrType têm MasVnrArea igual a 0.0?", todos_zero)

Todos os valores ausentes em MasVnrType têm MasVnrArea igual a 0.0? True


In [58]:
# Uma biblioteca que permite criar os gráficos atraves da interface muito parecida com o Tabeleu
# import pygwalker as pyg
#gwalker = pyg.walk(df_train)


In [59]:
df_train['LotFrontage'].value_counts()

LotFrontage
60.0     143
70.0      70
80.0      69
50.0      57
75.0      53
        ... 
137.0      1
141.0      1
38.0       1
140.0      1
46.0       1
Name: count, Length: 110, dtype: int64

In [60]:
df_train['LotFrontage'].isna().sum()

np.int64(259)

---

#### Tratando colunas Númericas

In [61]:
# Lista para armazenar colunas com pelo menos 1 nulo ou 0
colunas_problema = []

# Filtra colunas numéricas
colunas_numericas = df_train.select_dtypes(include='number').columns

for col in colunas_numericas:
    if df_train[col].isna().any():
        colunas_problema.append(col)

# Mostra o resultado
if colunas_problema:
    print("Colunas com valores nulos ou 0:")
    print(colunas_problema)
    print(len(colunas_problema))
else:
    print("Nenhuma coluna numérica com valores nulos ou 0.")

Colunas com valores nulos ou 0:
['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
3


In [62]:
from sklearn.impute import SimpleImputer
median_fill_cols = ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
# Imputar mediana para as outras
imputer = SimpleImputer(strategy='median')
df_train[median_fill_cols] = imputer.fit_transform(df_train[median_fill_cols])

In [63]:
import pandas as pd

def adicionar_numericas_no_df_clear(df_train, df_clear, alvo='SalePrice', id_col='Id'):
    # Seleciona colunas numéricas excluindo alvo e id
    numeric_cols = df_train.select_dtypes(include='number').columns.tolist()
    for col in [alvo, id_col]:
        if col in numeric_cols:
            numeric_cols.remove(col)
    
    # Extrai essas colunas do df_train
    df_numericas = df_train[numeric_cols]
    
    # Concatena no df_clear respeitando o índice
    df_clear_atualizado = pd.concat([df_clear, df_numericas], axis=1)
    
    return df_clear_atualizado

# Uso
df_clear = adicionar_numericas_no_df_clear(df_train, df_clear)

df_clear.shape

(1460, 272)

In [64]:
duplicated_cols = df_clear.columns[df_clear.columns.duplicated()]
print("Colunas duplicadas:", duplicated_cols.tolist())
df_clear = df_clear.loc[:, ~df_clear.columns.duplicated()]

Colunas duplicadas: ['HasFence']


In [65]:
# Supondo que você já tenha o pandas importado e o df_clear definido
import pandas as pd

# Salvar o DataFrame em um arquivo CSV
df_clear.to_csv('df_clear.csv', index=False)  # index=False para não salvar o índice no arquivo
